<a href="https://colab.research.google.com/github/faizaslam11/ECG_model_arrhythmia/blob/main/model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
import pandas as pd


In [9]:

# Load the cleaned data from your file
df = pd.read_csv('/content/all-samples-training-cleaned.csv')



In [11]:
# Extract features and labels from the processed DataFrame
features = df.drop('Arrhythmia', axis=1)
labels = df['Arrhythmia']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Reshape data to 3D for RNN compatibility (samples, timesteps, features)
X_train_rnn = np.expand_dims(X_train, axis=2)
X_test_rnn = np.expand_dims(X_test, axis=2)


In [13]:
# Build an RNN model using LSTM layers
rnn_model = models.Sequential([
    layers.LSTM(64, input_shape=(X_train_rnn.shape[1], 1), return_sequences=True),
    layers.LSTM(32),
    layers.Dense(16, activation='relu'),
    layers.Dense(len(np.unique(labels)), activation='softmax')  # Output layer for multi-class classification
])


In [14]:
# Compile the model
rnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the RNN model
rnn_history = rnn_model.fit(X_train_rnn, y_train, epochs=20, batch_size=32, validation_data=(X_test_rnn, y_test))


Epoch 1/20
74/74 [==============================] - 6s 23ms/step - loss: 0.6386 - accuracy: 0.9445 - val_loss: 0.1017 - val_accuracy: 0.9727
Epoch 2/20
74/74 [==============================] - 1s 10ms/step - loss: 0.0898 - accuracy: 0.9612 - val_loss: 0.0646 - val_accuracy: 0.9796
Epoch 3/20
74/74 [==============================] - 1s 10ms/step - loss: 0.0687 - accuracy: 0.9744 - val_loss: 0.0579 - val_accuracy: 0.9796
Epoch 4/20
74/74 [==============================] - 1s 10ms/step - loss: 0.0622 - accuracy: 0.9753 - val_loss: 0.0502 - val_accuracy: 0.9796
Epoch 5/20
74/74 [==============================] - 1s 11ms/step - loss: 0.0552 - accuracy: 0.9753 - val_loss: 0.0494 - val_accuracy: 0.9796
Epoch 6/20
74/74 [==============================] - 1s 13ms/step - loss: 0.0506 - accuracy: 0.9765 - val_loss: 0.0456 - val_accuracy: 0.9847
Epoch 7/20
74/74 [==============================] - 1s 16ms/step - loss: 0.0444 - accuracy: 0.9795 - val_loss: 0.0382 - val_accuracy: 0.9898
Epoch 8/20
74

In [15]:
# Evaluate model performance
rnn_loss, rnn_accuracy = rnn_model.evaluate(X_test_rnn, y_test)
print(f'Test Accuracy: {rnn_accuracy * 100:.2f}%')

19/19 [==============================] - 0s 8ms/step - loss: 0.0306 - accuracy: 0.9898
Test Accuracy: 98.98%
